In [71]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tqdm
import time
import random
from copy import deepcopy

import pickle

In [2]:
import pandas as pd
import numpy as np

In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


# Scrape

## Get Attractions

In [22]:
attraction_links = []

In [19]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [12]:
miss_links = []

In [20]:
soup = BeautifulSoup(driver.page_source)

### Finding Links Version 1

In [21]:
attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})

In [23]:
for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])

### Finding Links Version 2

In [30]:
soup = BeautifulSoup(driver.page_source)

In [31]:
attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))

In [35]:
# Will only work if there's the second type of link in the webpage

In [32]:
for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))

In [34]:
len(attraction_links)

30

### Finding Next Button

In [36]:
for i in soup.find_all('a', href=True):
    if i.text == '\nNext\n':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

In [38]:
for i in soup.find_all('a', href=True):
    if i.text == 'Next':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

<a href="/Attractions-g189400-Activities-oa30-Athens_Attica.html">Next</a>
<a href="/Attractions-g189400-Activities-oa30-Athens_Attica.html">Next</a>


### Code to run

In [ ]:
attraction_links = []
miss_links = []

In [ ]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [39]:
# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(12)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))


100%|██████████| 12/12 [00:38<00:00,  3.19s/it]


### Fix Attraction Links

In [44]:
attraction_links = list(set(attraction_links))

In [46]:
len(attraction_links)

576

In [51]:
#creating a test copy
t = deepcopy(attraction_links)

In [63]:
for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)

In [64]:
len(attraction_links)

397

In [68]:
fixed_links = []

In [69]:
for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

In [70]:
fixed_links

['https://www.tripadvisor.com/Attraction_Review-g189400-d12642863-Reviews-Gate_of_Medrese-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d2662541-Reviews-Museum_of_the_History_of_the_Greek_Costume_of_the_Lyceum_Club_of_Greek_Women-Athe.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d10237374-Reviews-Eleni_Marneri_Galerie-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d18979998-Reviews-Stoa_Avissinias-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d15151578-Reviews-Ikastikos_Kyklos_Sianti-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d10089083-Reviews-ELEYThERH_EKFRASH_Theater-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d1064133-Reviews-Temple_of_Hephaestus-Athens_Attica.html',
 'https://www.tripadvisor.com/Attraction_Review-g189400-d11691251-Reviews-Urban_Rail_Transport_S_A_Tramway-Athens_Attica.html',
 'https://www.tr

In [74]:
df = pd.DataFrame(fixed_links)

In [75]:
df.to_pickle('athens_links.pkl')

In [81]:
#convert dataframe to list, remember to call on column(0)
athens_links = df[0].tolist()

# Get Info From Attractions

In [84]:
athens_links[0]

'https://www.tripadvisor.com/Attraction_Review-g189400-d12642863-Reviews-Gate_of_Medrese-Athens_Attica.html'

In [85]:
driver.get(athens_links[0])

In [86]:
driver.find_element_by_xpath('//*[@id="HEADING"]').text

'Gate of Medrese'

In [88]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="HEADING"]').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/div[2]/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/span').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/span').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]').text

#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div').text

# def reviews(driver):
#     return 

In [89]:
find_name(driver)

'Gate of Medrese'

### Testing Selenium

In [90]:
attractions= []

In [91]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

## Click More on Comments

In [93]:
comments(driver)

"SoCalOregonian\nMurrieta, California\n3116931\nReviewed December 28, 2019\nFORMER MADRASA ENTRY PORTAL\nThis doorway is all that remains of an Islamic Madrasa from the Ottoman era. It operated in the 1700’s until converted to a prison. By 1915, all that remained was this doorway. This is located outside the Roman Agora across from the Tower of the Winds.\nShow less\nDate of experience: December 2019\nSee all 54 reviews by SoCalOregonian for Athens\nAsk SoCalOregonian about Gate of Medrese\nThank SoCalOregonian\nThis review is the subjective opinion of a TripAdvisor member and not of TripAdvisor LLC.\n450mikid\nThe Hague, The Netherlands\n5113\nReviewed November 3, 2018\nvia mobile\nIt's a door\nRich history (old ottoman Quran school) For a door It's located next to the Roman Agora Not as shown on this map\nDate of experience: November 2018\nThank 450mikid"

In [92]:
attractions

[{'name': 'Gate of Medrese',
  'no_things_to_do': '#225 of 319 things to do in Athens',
  'sights_landmarks': 'Sights & Landmarks, Points of Interest & Landmarks',
  'address': 'Pelopida, Athens 105 55, Greece',
  'summary_overview': '3.5',
  'num_reviews': '6 reviews',
  'about': 'About\nLOCAL WEATHER\nPowered by Weather Underground\nF°C°\n54°\n39°\nFeb\n58°\n42°\nMar\n65°\n48°\nApr',
  'popular_mentions': None,
  'comments': "SoCalOregonian\nMurrieta, California\n3116931\nReviewed December 28, 2019\nFORMER MADRASA ENTRY PORTAL\nThis doorway is all that remains of an Islamic Madrasa from the Ottoman era. It operated in the 1700’s until converted to a prison. By 1915, all that remained was this doorway. This is located outside the Roman Agora across from the Tower of the...More\nDate of experience: December 2019\nThank SoCalOregonian\n450mikid\nThe Hague, The Netherlands\n5113\nReviewed November 3, 2018\nvia mobile\nIt's a door\nRich history (old ottoman Quran school) For a door It's l

## Comments 

In [107]:
more_button = driver.find_element_by_class_name('ulBlueLinks')

In [108]:
more_button.click()

In [ ]:
driver.implicitly_wait(10)

## Click Next Comments

In [114]:
for i in driver.find_elements_by_class_name('next'):
    if i.text=='Next':
        next_button = i
        next_button.click()
        break

In [ ]:
<a data-page-number="2" data-offset="10" class="nav next ui_button primary" onclick="widgetEvCall('handlers.paginate', event, this); widgetEvCall('handlers.trackClick', event, this, 'pagination_next', '2');" href="/Attraction_Review-g189400-d198706-Reviews-or10-Acropolis-Athens_Attica.html">Next</a>

## For Loop Comments

In [118]:
comments(driver)

"11 - 20 of 19,660 reviews\nmartinedupont1504\nAntwerp, Belgium\n1010\nReviewed 1 week ago\nBreathtaking history and must see!\nThis is definitely a must-see. Try to book an archeologist guise in advance. Makes the event so rewarding. However, there are barricades around the major monuments... and lots of cranes due to restoration. It was still an awesome experience to view the Acropolis up close...More\nDate of experience: October 2019\n2  Thank martinedupont1504\nBengt H\n2\nReviewed 1 week ago\nvia mobile\nFantastic atmosphere\nThere are several reasons to visit this place. It gives closer details of things most visitors most probably have seen at the Acropolis. The explanation and recommendations of the ornaments of the Parthenon are especially impressive. The opening hours are long and there is a...More\nDate of experience: January 2020\nThank Bengt H\nOggieDee\nGreat Dunmow, United Kingdom\n15152\nReviewed 1 week ago\nvia mobile\nIn the foot steps of history!\nLoved visiting this

In [143]:
attraction_features['comments']= comments(driver)

In [127]:
attraction_features['comments']+=(comments(driver))

In [130]:
attraction_features['comments']+=''

In [144]:
len(attraction_features['comments'])

6343

In [145]:
attraction_features['comments']

'41 - 50 of 19,660 reviews\nHammadi99\n201\nReviewed 3 weeks ago\nvia mobile\nVisit\nIt was well organized site , reasonable tickets prices, easy to find and visit , there’s a guide available to whom how need.\nDate of experience: January 2020\n1  Thank Hammadi99\nwb250591\nCrete, Greece\n72\nReviewed 3 weeks ago\nvia mobile\nWell what can you say!\nSimply stunning of course , our first visit and the views and history just fill you with a sense of place and time! Not to be missed\nDate of experience: January 2020\nThank wb250591\nletsgomama\nSummit, New Jersey\n9346\nReviewed 3 weeks ago\nYou have to do it. And you can. If you are brave. But it is so worth it!\nI am mobility impaired. I thought the Acropolis would be one of places that would be off limits to me. We were smart enough to book the Viator full day Athens private tour, and our wonderful driver knew exactly where to take me and who to talk to. (There is a discount for disabled people and their one companion.) He took me as c

In [147]:
len(attraction_features['comments'])

28603

In [148]:
attraction_features['comments']

'41 - 50 of 19,660 reviews\nHammadi99\n201\nReviewed 3 weeks ago\nvia mobile\nVisit\nIt was well organized site , reasonable tickets prices, easy to find and visit , there’s a guide available to whom how need.\nDate of experience: January 2020\n1  Thank Hammadi99\nwb250591\nCrete, Greece\n72\nReviewed 3 weeks ago\nvia mobile\nWell what can you say!\nSimply stunning of course , our first visit and the views and history just fill you with a sense of place and time! Not to be missed\nDate of experience: January 2020\nThank wb250591\nletsgomama\nSummit, New Jersey\n9346\nReviewed 3 weeks ago\nYou have to do it. And you can. If you are brave. But it is so worth it!\nI am mobility impaired. I thought the Acropolis would be one of places that would be off limits to me. We were smart enough to book the Viator full day Athens private tour, and our wonderful driver knew exactly where to take me and who to talk to. (There is a discount for disabled people and their one companion.) He took me as c

In [146]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()

    time.sleep(random.choice(sequence))

    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass


    #get comments
    # add to dictionary

    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''

    time.sleep(random.choice(sequence))

## Adding to Selenium Code

In [149]:
sequence = [x/10 for x in range(20, 30)]

In [150]:
attractions = []

In [159]:
test = athens_links[:20]

In [160]:
len(test)

20

In [ ]:
for i in tqdm(test):
    driver.get(i)
    attraction_features = {}
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()

    except:
        pass

    # add comments to dictionary
    driver.implicitly_wait(5)
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None

    try:
        #loop through 1st 3 pages if they have a next
        for next_comments in range(4):
            if driver.find_element_by_class_name('next')== None:
                break
            else:
                for i in driver.find_elements_by_class_name('next'):
                    if i.text=='Next':
                        next_button = i
                        next_button.click()

            time.sleep(random.choice(sequence))

            try:
                more_button = driver.find_element_by_class_name('ulBlueLinks')
                more_button.click()
                time.sleep(random.choice(sequence))

            except:
                pass


            #get comments
            # add to dictionary

            driver.implicitly_wait(5)
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''

            time.sleep(random.choice(sequence))
        
        except:
            pass

    #add to list of dictionaries
    attractions.append(attraction_features)



In [ ]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

In [ ]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()
    
    time.sleep(random.choice(sequence))
    
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass
        
    
    #get comments
    # add to dictionary
    
    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''
        
    time.sleep(random.choice(sequence))